In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from qiskit.tools.jupyter import *
from qiskit import IBMQ
from qiskit import assemble
from qiskit.tools.monitor import job_monitor

from qiskit import pulse
from qiskit.pulse import Play
from qiskit.pulse import pulse_lib

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

In [2]:
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns")    # The configuration returns dt in seconds, so multiply by
                                        # 1e9 to get nanoseconds
    
backend_defaults = backend.defaults()

Sampling time: 0.2222222222222222 ns


In [3]:
# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# We will find the qubit frequency for the following qubit.
qubit = 0

# The sweep will be centered around the estimated qubit frequency.
center_frequency_Hz = backend_defaults.qubit_freq_est[qubit]        # The default frequency is given in Hz
                                                                    # warning: this will change in a future release
print(f"Qubit {qubit} has an estimated frequency of {center_frequency_Hz / GHz} GHz.")

# scale factor to remove factors of 10 from the data
scale_factor = 1e-14

# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

freq_min = None
freq_max = None
drive_sigma_us = None
drive_samples_us = None
drive_sigma = None
drive_samples = None
drive_amp = None
drive_pulse = None
job = None
rough_qubit_frequency = None

Qubit 0 has an estimated frequency of 4.974438151218951 GHz.


In [4]:
num_rabi_points = 50

drive_amp_min = 0
drive_amp_max = 0.75
pi_pulse = None
job2 = None

def baseline_remove(values):
    return np.array(values) - np.mean(values)

In [5]:
def rabi_exp(drive_min, drive_max, num_points, rough_qubit_freq):
    global num_rabi_points
    global drive_amp_min
    global drive_amp_max
    global pi_pulse
    global job2
    global rough_qubit_frequency
    
    rough_qubit_frequency = rough_qubit_freq * GHz
    
    num_rabi_points = num_points
    drive_amp_min = drive_min
    drive_amp_max = drive_max
    
    drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)
    
    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    rabi_schedules = []
    for drive_amp in drive_amps:
        rabi_pulse = pulse_lib.gaussian(duration=drive_samples, amp=drive_amp, 
                                    sigma=drive_sigma, name=f"Rabi drive amplitude = {drive_amp}")
        this_schedule = pulse.Schedule(name=f"Rabi drive amplitude = {drive_amp}")
        this_schedule += Play(rabi_pulse, drive_chan)
        
    meas_map_idx = None
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
        
    this_schedule += measure << this_schedule.duration
    rabi_schedules.append(this_schedule)
    
    rabi_schedules[-1].draw(label=True, scaling=1.0)
    
    num_shots_per_point = 1024

    rabi_experiment_program = assemble(rabi_schedules,
                                   backend=backend,
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_point,
                                   schedule_los=[{drive_chan: rough_qubit_frequency}]
                                                * num_rabi_points)
    
    # print(job.job_id())
    job2 = backend.run(rabi_experiment_program)
    job_monitor(job2)
    
    rabi_results = job2.result(timeout=120)
    
    rabi_values = []
    for i in range(num_rabi_points):
        rabi_values.append(rabi_results.get_memory(i)[qubit]*scale_factor)

    rabi_values = np.real(baseline_remove(rabi_values))

    plt.xlabel("Drive amp [a.u.]")
    plt.ylabel("Measured signal [a.u.]")
    plt.scatter(drive_amps, rabi_values, color='black') # plot real part of Rabi values
    plt.show()
    
    fit_params, y_fit = fit_function(drive_amps,
                                 rabi_values, 
                                 lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),
                                 [3, 0.1, 0.5, 0])

    plt.scatter(drive_amps, rabi_values, color='black')
    plt.plot(drive_amps, y_fit, color='red')

    drive_period = fit_params[2] # get period of rabi oscillation

    plt.axvline(drive_period/2, color='red', linestyle='--')
    plt.axvline(drive_period, color='red', linestyle='--')
    plt.annotate("", xy=(drive_period, 0), xytext=(drive_period/2,0), arrowprops=dict(arrowstyle="<->", color='red'))
    plt.annotate("$\pi$", xy=(drive_period/2-0.03, 0.1), color='red')

    plt.xlabel("Drive amp [a.u.]", fontsize=15)
    plt.ylabel("Measured signal [a.u.]", fontsize=15)
    plt.show()
    
    pi_amp = abs(drive_period / 2)
    print(f"Pi Amplitude = {pi_amp}")
    
    pi_pulse = pulse_lib.gaussian(duration=drive_samples,
                              amp=pi_amp, 
                              sigma=drive_sigma,
                              name='pi_pulse')

In [6]:
def spectr_exp(frequency_min, frequency_max, sigma, amp):
    global freq_min
    global freq_max
    global drive_sigma_us
    global drive_samples_us
    global drive_sigma
    global drive_samples
    global drive_amp
    global drive_pulse
    global job1
    global rough_qubit_frequency

    frequency_span_Hz = (frequency_max - frequency_min) * MHz
    # in steps of 1 MHz.
    frequency_step_Hz = 0.5 * MHz

    frequency_min = frequency_min * GHz
    frequency_max = frequency_max * GHz
    frequencies_GHz = np.arange(frequency_min / GHz, 
                            frequency_max / GHz, 
                            frequency_step_Hz / GHz)
    
    print(f"The sweep will go from {frequency_min / GHz} GHz to {frequency_max / GHz} GHz \
    in steps of {frequency_step_Hz / MHz} MHz.")
    
    # Drive pulse parameters (us = microseconds)
    drive_sigma_us = sigma                     # This determines the actual width of the gaussian
    drive_samples_us = sigma * 8               # This is a truncating parameter, because gaussians don't have 
                                               # a natural finite length

    drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)       # The width of the gaussian in units of dt
    drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)   # The truncating parameter in units of dt
    drive_amp = amp
    # Drive pulse samples
    drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                     sigma=drive_sigma,
                                     amp=drive_amp,
                                     name='freq_sweep_excitation_pulse')

    meas_map_idx = None
    for i, measure_group in enumerate(backend_config.meas_map):
        if qubit in measure_group:
            meas_map_idx = i
            break
    assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in the meas_map!"
    
    inst_sched_map = backend_defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=backend_config.meas_map[meas_map_idx])
    
    drive_chan = pulse.DriveChannel(qubit)
    meas_chan = pulse.MeasureChannel(qubit)
    acq_chan = pulse.AcquireChannel(qubit)
    
    schedule = pulse.Schedule(name='Frequency sweep')
    schedule += Play(drive_pulse, drive_chan)
    schedule += measure << schedule.duration

    frequencies_Hz = frequencies_GHz*GHz
    schedule_frequencies = [{drive_chan: freq} for freq in frequencies_Hz]
    #schedule.draw(label=True, scaling=0.8)
    
    num_shots_per_frequency = 1024
    frequency_sweep_program = assemble(schedule,
                                   backend=backend, 
                                   meas_level=1,
                                   meas_return='avg',
                                   shots=num_shots_per_frequency,
                                   schedule_los=schedule_frequencies)
    
    job1 = backend.run(frequency_sweep_program)
    job_monitor(job1)
    frequency_sweep_results = job1.result(timeout=120)
    
    

    sweep_values = []
    for i in range(len(frequency_sweep_results.results)):
        res = frequency_sweep_results.get_memory(i)*scale_factor
        sweep_values.append(res[qubit])

    plt.scatter(frequencies_GHz, np.real(sweep_values), color='black')
    plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])
    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Measured signal [a.u.]")
    plt.show()
    
    fit_params, y_fit = fit_function(frequencies_GHz,
                                 np.real(sweep_values), 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [5, 4.975, 1, 3] # initial parameters for curve_fit
                                )
    
    plt.scatter(frequencies_GHz, np.real(sweep_values), color='black')
    plt.plot(frequencies_GHz, y_fit, color='red')
    plt.xlim([min(frequencies_GHz), max(frequencies_GHz)])

    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Measured Signal [a.u.]")
    plt.show()
    
    A, rough_qubit_frequency, B, C = fit_params
    rough_qubit_frequency = rough_qubit_frequency*GHz # make sure qubit freq is in Hz
    print(f"Based on the parameters input, the new qubit frequency is " + str(round(rough_qubit_frequency/GHz, 5)) +" GHz.")

In [7]:
interact_manual(spectr_exp,
    frequency_min = widgets.FloatSlider(
    value=4.970,
    min=4.955,
    max=4.995,
    step=0.001,
    description='Min Freq:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    ),
    frequency_max = widgets.FloatSlider(
    value=4.980,
    min=4.955,
    max=4.995,
    step=0.001,
    description='Max Freq:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    ),
    sigma = widgets.FloatSlider(
    value=0.075,
    min=0.06,
    max=0.09,
    step=0.005,
    description='Sigma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    ),
    amp = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.5,
    step=0.05,
    description='Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ))
interact_manual(rabi_exp,
    drive_min = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.75,
    step=0.05,
    description='Min Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ),
    drive_max = widgets.FloatSlider(
    value=0.75,
    min=-0,
    max=1,
    step=0.05,
    description='Max Amp:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    ),
    num_points = widgets.IntSlider(
    value=50,
    min=10,
    max=100,
    step=0.05,
    description='Points:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    ),
    rough_qubit_freq = widgets.BoundedFloatText(
    value=4.974,
    min=4.955,
    max=4.995,
    step=0.0001,
    description='Frequency
    disabled=False
))

interactive(children=(FloatSlider(value=4.97, continuous_update=False, description='Minimum Frequency:', max=4…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Min Drive Amp:', max=0.75, …

<function __main__.rabi_exp(drive_min, drive_max, num_points, rough_qubit_freq)>